In [4]:
import tkinter as tk
import random
import math

# -------------------- GLOBAL CONSTANTS --------------------
WIDTH = 600
HEIGHT = 400
BALL_RADIUS = 20
PAD_WIDTH = 10
PAD_HEIGHT = 80

BALL_SPEED = 5
PADDLE_SPEED = 15

# -------------------- GAME CLASS --------------------
class PongGame:
    def __init__(self, master):
        self.master = master
        master.title("Classic Pong Game")

        # Canvas setup
        self.canvas = tk.Canvas(master, width=WIDTH, height=HEIGHT, bg="black")
        self.canvas.pack()

        # Draw middle line
        self.canvas.create_line(WIDTH/2, 0, WIDTH/2, HEIGHT, fill="white")

        # Paddle 1 (Left)
        self.paddle1 = self.canvas.create_rectangle(0, HEIGHT/2 - PAD_HEIGHT/2,
                                                    PAD_WIDTH, HEIGHT/2 + PAD_HEIGHT/2,
                                                    fill="white")

        # Paddle 2 (Right)
        self.paddle2 = self.canvas.create_rectangle(WIDTH - PAD_WIDTH, HEIGHT/2 - PAD_HEIGHT/2,
                                                    WIDTH, HEIGHT/2 + PAD_HEIGHT/2,
                                                    fill="blue")

        # Ball
        self.ball = self.canvas.create_oval(WIDTH/2 - BALL_RADIUS, HEIGHT/2 - BALL_RADIUS,
                                            WIDTH/2 + BALL_RADIUS, HEIGHT/2 + BALL_RADIUS,
                                            fill="orange")

        # Scores
        self.score1 = 0
        self.score2 = 0
        self.score_text = self.canvas.create_text(WIDTH/2 - 50, 30, text="0", fill="white", font=("Arial", 20))
        self.score_text2 = self.canvas.create_text(WIDTH/2 + 50, 30, text="0", fill="blue", font=("Arial", 20))

        # Ball velocity
        self.ball_vel = [random.choice([-BALL_SPEED, BALL_SPEED]),
                         random.choice([-BALL_SPEED, BALL_SPEED])]

        # Paddle velocities
        self.paddle1_vel = 0
        self.paddle2_vel = 0

        # Bind controls
        master.bind("<KeyPress>", self.keydown)
        master.bind("<KeyRelease>", self.keyup)

        # Restart button
        self.restart_button = tk.Button(master, text="Restart", command=self.restart, width=10)
        self.restart_button.pack(pady=10)

        # Start game loop
        self.game_loop()

    # -------------------- GAME LOGIC --------------------
    def move_ball(self):
        self.canvas.move(self.ball, self.ball_vel[0], self.ball_vel[1])
        ball_coords = self.canvas.coords(self.ball)
        left, top, right, bottom = ball_coords

        # Top and bottom wall collisions
        if top <= 0 or bottom >= HEIGHT:
            self.ball_vel[1] = -self.ball_vel[1]

        # Paddle collision
        if self.hit_paddle(self.paddle1, ball_coords):
            self.ball_vel[0] = abs(self.ball_vel[0])  # bounce right
            self.increase_speed()

        elif self.hit_paddle(self.paddle2, ball_coords):
            self.ball_vel[0] = -abs(self.ball_vel[0])  # bounce left
            self.increase_speed()

        # Left gutter (Player 2 scores)
        if left <= 0:
            self.score2 += 1
            self.update_scores()
            self.spawn_ball(right=True)

        # Right gutter (Player 1 scores)
        elif right >= WIDTH:
            self.score1 += 1
            self.update_scores()
            self.spawn_ball(right=False)

    def hit_paddle(self, paddle, ball_coords):
        px1, py1, px2, py2 = self.canvas.coords(paddle)
        bx1, by1, bx2, by2 = ball_coords
        return (bx2 >= px1 and bx1 <= px2) and (by2 >= py1 and by1 <= py2)

    def move_paddles(self):
        # Left paddle
        self.canvas.move(self.paddle1, 0, self.paddle1_vel)
        p1_coords = self.canvas.coords(self.paddle1)
        if p1_coords[1] < 0:
            self.canvas.move(self.paddle1, 0, -p1_coords[1])
        elif p1_coords[3] > HEIGHT:
            self.canvas.move(self.paddle1, 0, HEIGHT - p1_coords[3])

        # Right paddle
        self.canvas.move(self.paddle2, 0, self.paddle2_vel)
        p2_coords = self.canvas.coords(self.paddle2)
        if p2_coords[1] < 0:
            self.canvas.move(self.paddle2, 0, -p2_coords[1])
        elif p2_coords[3] > HEIGHT:
            self.canvas.move(self.paddle2, 0, HEIGHT - p2_coords[3])

    def increase_speed(self):
        self.ball_vel[0] *= 1.1
        self.ball_vel[1] *= 1.1

    def update_scores(self):
        self.canvas.itemconfig(self.score_text, text=str(self.score1))
        self.canvas.itemconfig(self.score_text2, text=str(self.score2))

    def spawn_ball(self, right=True):
        # Reset ball to center
        self.canvas.coords(self.ball, WIDTH/2 - BALL_RADIUS, HEIGHT/2 - BALL_RADIUS,
                           WIDTH/2 + BALL_RADIUS, HEIGHT/2 + BALL_RADIUS)
        self.ball_vel = [random.choice([BALL_SPEED, -BALL_SPEED]),
                         random.choice([BALL_SPEED, -BALL_SPEED])]
        if not right:
            self.ball_vel[0] *= -1

    # -------------------- CONTROLS --------------------
    def keydown(self, event):
        key = event.keysym
        if key == 'w':
            self.paddle1_vel = -PADDLE_SPEED
        elif key == 's':
            self.paddle1_vel = PADDLE_SPEED
        elif key == 'Up':
            self.paddle2_vel = -PADDLE_SPEED
        elif key == 'Down':
            self.paddle2_vel = PADDLE_SPEED

    def keyup(self, event):
        key = event.keysym
        if key in ['w', 's']:
            self.paddle1_vel = 0
        elif key in ['Up', 'Down']:
            self.paddle2_vel = 0

    def restart(self):
        self.score1 = 0
        self.score2 = 0
        self.update_scores()
        self.spawn_ball(random.choice([True, False]))

    # -------------------- GAME LOOP --------------------
    def game_loop(self):
        self.move_ball()
        self.move_paddles()
        self.master.after(30, self.game_loop)

# -------------------- RUN THE GAME --------------------
root = tk.Tk()
game = PongGame(root)
root.mainloop()